In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/MyDrive')
!ls

Mounted at /content/drive/
'4032 DBSystem Proj1'	     'Grp Project Report.gdoc'
 4041ML			     'Jinling High School Certificate.pdf'
'Colab Notebooks'	      NLP-Assignment.gdoc
'CZ4031 P2 Report.gdoc'      'Zhang Yuehan 2.1.pdf'
'Degree Audit Yr4 S1.pdf'    'Zhang Yuehan护照.pdf'
'Final Project Report.gdoc'


In [ ]:
cd 4041ML/output/feature_eng_final

/content/drive/MyDrive/4041ML/output/feature_eng_final


In [ ]:
import numpy as np
import pandas as pd

## Data

In [ ]:
train_X = pd.read_csv('train_X.csv')
train_y = pd.read_csv('train_y.csv')

In [ ]:
train_X.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,...,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,load_of_teachers_preschool_per_teacher,child_on_acc_pre_school,load_of_teachers_school_per_teacher,students_state_oneshift,modern_education_share,old_education_build_share,provision_doctors,provision_nurse,load_on_doctors,power_clinics,hospital_beds_available_per_cap,hospital_bed_occupancy_per_year,provision_retail_space_sqm,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm,yearmonth,yearweek,year,month_of_year,week_of_year,day_of_week,ratio_life_full,ratio_kitch_life,ratio_kitch_full,full_live_diff,ratio_floor_max_floor,floor_from_top,age_of_building,apartment_name
0,43.0,27.0,4.0,13.0,2.6,1975.6,1.0,7.2,2.2,0.0,9.0,6.407578e+06,155572.0,0.189727,0.000070,9576.0,5001.0,5.0,10309.0,11065.0,5.0,0.0,240.0,1.0,0.0,7.0,3.0,0.0,0.0,16.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86206.0,...,41.80,36.77,29.07,793.319561,0.0,1391.710938,89.0495,3.0,3.0,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0,201108,201133,2011,8,33,5,0.627907,0.266667,0.167442,16.0,0.307692,9.0,35.4,0
1,43.0,29.0,2.0,7.6,2.6,1963.2,1.4,6.2,2.4,0.0,129.0,4.808270e+06,101708.0,0.112560,0.118537,5879.0,1463.0,4.0,6207.0,5580.0,7.0,0.0,1183.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,101982.0,...,41.80,36.77,29.07,793.319561,0.0,1391.710938,89.0495,3.0,3.0,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0,201108,201134,2011,8,34,5,0.674419,0.213793,0.144186,14.0,0.263158,5.6,47.8,1
2,89.0,50.0,9.0,18.0,2.6,2003.0,3.4,8.2,2.8,0.0,65.0,1.258354e+07,178473.0,0.194703,0.069753,13087.0,6839.0,9.0,13670.0,17063.0,10.0,0.0,1128.6,1.0,0.0,17.0,6.0,0.0,0.0,11.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21155.0,...,44.25,37.73,30.63,793.319561,0.0,1391.710938,89.0495,3.0,3.0,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0,201109,201135,2011,9,35,3,0.561798,0.164000,0.092135,39.0,0.500000,9.0,8.0,2
3,77.0,77.0,4.0,6.2,2.0,1949.8,3.4,7.4,2.6,0.0,6.0,8.398461e+06,108171.0,0.015234,0.037316,5706.0,3240.0,7.0,6748.0,7770.0,9.0,0.0,562.0,4.0,2.0,25.0,2.0,0.0,0.0,10.0,93.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,28179.0,...,44.25,37.73,30.63,793.319561,0.0,1391.710938,89.0495,3.0,3.0,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0,201109,201136,2011,9,36,0,1.000000,0.096104,0.096104,0.0,0.645161,2.2,61.2,3
4,44.0,44.0,5.0,16.2,1.0,1979.6,3.0,8.6,2.4,0.0,9.0,6.407578e+06,155572.0,0.189727,0.000070,9576.0,5001.0,5.0,10309.0,11065.0,5.0,0.0,240.0,1.0,0.0,7.0,3.0,0.0,0.0,16.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86206.0,...,44.25,37.73,30.63,793.319561,0.0,1391.710938,89.0495,3.0,3.0,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0,201109,201136,2011,9,36,4,1.000000,0.195455,0.195455,0.0,0.308642,11.2,31.4,4


In [ ]:
train_y.head()

,0
0,15.581952
1,15.555977
2,16.388123
3,16.608603
4,14.508658


In [ ]:
np.array(train_y)

array([[15.58195239],
       [15.55597691],
       [16.38812286],
       ...,
       [17.03438642],
       [16.41820032],
       [15.53827733]])

In [ ]:
print(np.any(np.isnan(train_X)))
np.all(np.isfinite(train_X))

False


True

In [ ]:
print(np.any(np.isnan(train_y)))
np.all(np.isfinite(train_y))

False


True

## Grid Search


In [ ]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(max_iter=100)

In [ ]:
parameter_space = {
    'hidden_layer_sizes': [(100,), (25,3), (20,3), (25,2)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5)
clf.fit(train_X, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, error_score=nan,
             estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                    batch_size='auto', beta_1=0.9, beta_2=0.999,
                                    early_stopping=False, epsilon=1e-08,
                                    hidden_layer_sizes=(100,),
                                    learning_rate='constant',
                                    learning_rate_init=0.001, max_fun=15000,
                                    max_iter=100, momentum=0.9,
                                    n_iter_no_change=10,
                                    nesterovs_momentum=True, power_t=0.5,
                                    random_state=...
                                    solver='adam', tol=0.0001,
                                    validation_fraction=0.1, verbose=False,
                                    warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu']

In [ ]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'solver': 'adam'}
-0.020 (+/-0.074) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'solver': 'adam'}
-0.000 (+/-0.069) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'solver': 'adam'}
-0.023 (+/-0.057) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (25, 3), 'learning_rate': 'constant', 'solver': 'adam'}
-0.022 (+/-0.060) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (25, 3), 'learning_rate': 'adaptive', 'solver': 'adam'}
-0.025 (+/-0.055) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 3), 'learning_rate': 'constant', 'solver': 'adam'}
-0.017 (+/-0.058) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 3), 'learning_rate': 'adaptive', 'solver': 'adam'}
-0.029 (+/-0.058) for

In [ ]:
import pickle
import joblib

# save model
if not os.path.exists('cv_models'):
    os.makedirs('cv_models')
joblib.dump(clf, 'cv_models/mlp.pkl')

['cv_models/mlp.pkl']

## vif 75

In [ ]:
with open('remain_features.pkl', 'rb') as f:
    features = pickle.load(f)
    print(features)
print(len(features))

['material' 'product_type' 'indust_part'
 'school_education_centers_top_20_raion' 'hospital_beds_raion'
 'healthcare_centers_raion' 'university_top_20_raion'
 'additional_education_raion' 'culture_objects_top_25'
 'thermal_power_plant_raion' 'incineration_raion' 'oil_chemistry_raion'
 'radiation_raion' 'railroad_terminal_raion' 'big_market_raion'
 'nuclear_reactor_raion' 'detention_facility_raion' '16_29_male'
 'build_count_block' 'build_count_frame' 'build_count_monolith'
 'build_count_foam' 'build_count_before_1920' 'build_count_1971-1995'
 'ID_metro' 'kindergarten_km' 'park_km' 'green_zone_km'
 'ID_railroad_station_avto' 'public_transport_station_min_walk' 'water_km'
 'water_1line' 'ID_big_road1' 'big_road1_1line' 'ID_big_road2'
 'railroad_km' 'railroad_1line' 'ID_railroad_terminal' 'fitness_km'
 'catering_km' 'green_part_500' 'prom_part_500' 'office_sqm_500'
 'trc_count_500' 'trc_sqm_500' 'cafe_count_500_na_price'
 'cafe_count_500_price_1000' 'cafe_count_500_price_4000'
 'cafe_coun

In [ ]:
train_X_selected = train_X[[c for c in train_X.columns if c in features]]

In [ ]:
from sklearn.neural_network import MLPRegressor
mlp_selected = MLPRegressor(max_iter=100)

In [ ]:
from sklearn.model_selection import GridSearchCV

clf_selected = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5)
clf_selected.fit(train_X_selected, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                    batch_size='auto', beta_1=0.9, beta_2=0.999,
                                    early_stopping=False, epsilon=1e-08,
                                    hidden_layer_sizes=(100,),
                                    learning_rate='constant',
                                    learning_rate_init=0.001, max_fun=15000,
                                    max_iter=100, momentum=0.9,
                                    n_iter_no_change=10,
                                    nesterovs_momentum=True, power_t=0.5,
                                    random_state=...
                                    solver='adam', tol=0.0001,
                                    validation_fraction=0.1, verbose=False,
                                    warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu']

In [ ]:
# Best paramete set
print('Best parameters found:\n', clf_selected.best_params_)

# All results
means_selected = clf_selected.cv_results_['mean_test_score']
stds_selected = clf_selected.cv_results_['std_test_score']
for mean, std, params in zip(means_selected, stds_selected, clf_selected.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'solver': 'adam'}
-0.034 (+/-0.165) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'solver': 'adam'}
-0.094 (+/-0.245) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'solver': 'adam'}
-0.029 (+/-0.059) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (25, 3), 'learning_rate': 'constant', 'solver': 'adam'}
-0.030 (+/-0.060) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (25, 3), 'learning_rate': 'adaptive', 'solver': 'adam'}
-0.030 (+/-0.061) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 3), 'learning_rate': 'constant', 'solver': 'adam'}
-0.030 (+/-0.061) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 3), 'learning_rate': 'adaptive', 'solver': 'adam'}
-0.022 (+/-0.070) for {

In [ ]:
# save model
if not os.path.exists('cv_models'):
    os.makedirs('cv_models')
joblib.dump(clf_selected, 'cv_models/mlp_selected.pkl')

['cv_models/mlp_selected.pkl']

## Train & Test

In [ ]:
mdl = joblib.load('cv_models/mlp.pkl')
mdl_selected = joblib.load('cv_models/mlp_selected.pkl')

mlp = MLPRegressor(**mdl.best_params_)
mlp_selected = MLPRegressor(**mdl_selected.best_params_)

mlp.fit(train_X, train_y)
mlp_selected.fit(train_X_selected, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='tanh', alpha=0.05, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [ ]:
from sklearn.metrics import mean_squared_log_error
tst_mlp = np.sqrt(mean_squared_log_error(train_y, mlp.predict(train_X)))
tst_mlp_selected = np.sqrt(mean_squared_log_error(train_y, mlp_selected.predict(train_X_selected)))

print(f'original feature: {tst_mlp}')
print(f'selected feature: {tst_mlp_selected}')

original feature: 0.03917467547827968
selected feature: 0.039405488622935796


In [ ]:
test_X = pd.read_csv('test_X_1_clean.csv')
test_X.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,...,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,load_of_teachers_preschool_per_teacher,child_on_acc_pre_school,load_of_teachers_school_per_teacher,students_state_oneshift,modern_education_share,old_education_build_share,provision_doctors,provision_nurse,load_on_doctors,power_clinics,hospital_beds_available_per_cap,hospital_bed_occupancy_per_year,provision_retail_space_sqm,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm,yearmonth,yearweek,year,month_of_year,week_of_year,day_of_week,ratio_life_full,ratio_kitch_life,ratio_kitch_full,full_live_diff,ratio_floor_max_floor,floor_from_top,age_of_building,apartment_name
0,39.0,20.700,2.0,9.0,1.0,1998.0,1.0,8.9,3.0,0.0,38.0,2.615514e+07,178264.0,0.137846,0.041116,14080.0,11926.0,11.0,14892.0,24750.0,13.0,1.0,1848.2,1.0,0.0,13.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102618.0,...,49.35,39.62,30.98,863.394794,5.0,1573.650623,97.8484,2.0,2.0,61.0,90.8,6899.930464,41.9,707.0,286.0,794.0,271.0,10805.0,627.0,0.45888,1440.0,463938.0,28.2,76.2,46080.0,234576.9,201507,201527,2015,7,27,2,0.530769,0.429952,0.228205,18.300,0.222222,7.0,17.000000,0
1,79.2,49.654,8.0,17.0,1.0,0.0,3.0,1.0,1.0,1.0,102.0,2.553630e+07,4001.0,0.496315,0.007122,275.0,11926.0,0.0,264.0,24750.0,0.0,0.0,2300.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17790.0,...,49.35,39.62,30.98,863.394794,5.0,1573.650623,97.8484,2.0,2.0,61.0,90.8,6899.930464,41.9,707.0,286.0,794.0,271.0,10805.0,627.0,0.45888,1440.0,463938.0,28.2,76.2,46080.0,234576.9,201507,201527,2015,7,27,2,0.626944,0.020139,0.012626,29.546,0.470588,9.0,2015.000000,1
2,40.5,25.100,3.0,5.0,2.0,1960.0,2.0,4.8,2.0,0.0,84.0,9.946335e+06,139322.0,0.065409,0.225825,6400.0,2232.0,7.0,6558.0,7966.0,7.0,0.0,1548.0,3.0,0.0,13.0,0.0,0.0,0.0,2.0,7.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,36154.0,...,49.35,39.62,30.98,863.394794,5.0,1573.650623,97.8484,2.0,2.0,61.0,90.8,6899.930464,41.9,707.0,286.0,794.0,271.0,10805.0,627.0,0.45888,1440.0,463938.0,28.2,76.2,46080.0,234576.9,201507,201527,2015,7,27,2,0.619753,0.191235,0.118519,15.400,0.600000,2.0,55.000000,2
3,62.8,36.000,17.0,17.0,1.0,2016.0,2.0,62.8,3.0,1.0,104.0,2.149409e+07,7122.0,0.262459,0.017647,489.0,903.0,0.0,469.0,7788.0,0.0,0.0,176.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9553.0,...,49.35,39.62,30.98,863.394794,5.0,1573.650623,97.8484,2.0,2.0,61.0,90.8,6899.930464,41.9,707.0,286.0,794.0,271.0,10805.0,627.0,0.45888,1440.0,463938.0,28.2,76.2,46080.0,234576.9,201507,201527,2015,7,27,2,0.573248,1.000000,1.000000,26.800,1.000000,0.0,-1053.057097,3
4,40.0,40.000,17.0,17.0,1.0,0.0,1.0,1.0,1.0,1.0,102.0,2.553630e+07,4001.0,0.496315,0.007122,275.0,11926.0,0.0,264.0,24750.0,0.0,0.0,2300.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17790.0,...,49.35,39.62,30.98,863.394794,5.0,1573.650623,97.8484,2.0,2.0,61.0,90.8,6899.930464,41.9,707.0,286.0,794.0,271.0,10805.0,627.0,0.45888,1440.0,463938.0,28.2,76.2,46080.0,234576.9,201507,201527,2015,7,27,2,1.000000,0.025000,0.025000,0.000,1.000000,0.0,2015.000000,4


In [ ]:
test_X.shape[0]

7662

In [ ]:
print(test_X.isnull().values.any())

count = np.isinf(test_X["ratio_floor_max_floor"]).values.sum()
print("It contains " + str(count) + " infinite values")

False
It contains 233 infinite values


In [ ]:
test_X = test_X.replace([np.inf, -np.inf], np.nan)
test_X = test_X.fillna(0.0)

In [ ]:
print(test_X.isnull().values.any())

count = np.isinf(test_X["ratio_floor_max_floor"]).values.sum()
print("It contains " + str(count) + " infinite values")

False
It contains 0 infinite values


In [ ]:
test_X_selected = test_X[[c for c in test_X.columns if c in features]]

In [ ]:
prdct = mlp.predict(test_X)
prdct

array([15.42945121, 15.42945121, 15.50990233, ..., 15.69094436,
       15.50990233, 15.69094436])

In [ ]:
prdct_selected = mlp_selected.predict(test_X_selected)
prdct_selected

array([15.53682689, 15.53682689, 15.53682689, ..., 15.7574496 ,
       15.66054548, 15.83683119])

In [ ]:
predict = np.expm1(prdct)
predict.reshape(1,7662)

array([5022563.44015805, 5022563.44015805, 5443333.17297839, ...,
       6523647.47640257, 5443333.17297843, 6523647.47640252])

In [ ]:
predict_selected = np.expm1(prdct_selected)
predict_selected


array([5591883.42626988, 5591883.42626988, 5591883.42626988, ...,
       6972256.33789628, 6328319.80470936, 7548285.86881576])

In [ ]:
id = np.arange(30474,38136,1)
id.reshape(1,7662)

array([[30474, 30475, 30476, ..., 38133, 38134, 38135]])

In [ ]:
submission = np.column_stack((id, predict))
submission_selected = np.column_stack((id, predict_selected))

In [ ]:
submission

array([[  30474.        , 5022563.44015805],
       [  30475.        , 5022563.44015805],
       [  30476.        , 5443333.17297839],
       ...,
       [  38133.        , 6523647.47640257],
       [  38134.        , 5443333.17297843],
       [  38135.        , 6523647.47640252]])

In [ ]:
submission_selected

array([[  30474.        , 5591883.42626988],
       [  30475.        , 5591883.42626988],
       [  30476.        , 5591883.42626988],
       ...,
       [  38133.        , 6972256.33789628],
       [  38134.        , 6328319.80470936],
       [  38135.        , 7548285.86881576]])

In [ ]:
df = pd.DataFrame(data=submission, columns=["id", "price_doc"])
df['id'] = df['id'].astype('int64')
df.head()

,id,price_doc
0,30474,5.022563e+06
1,30475,5.022563e+06
2,30476,5.443333e+06
3,30477,5.008446e+06
4,30478,5.022563e+06


In [ ]:
df_selected = pd.DataFrame(data=submission_selected, columns=["id", "price_doc"])
df_selected['id'] = df_selected['id'].astype('int64')
df_selected.head()

,id,price_doc
0,30474,5.591883e+06
1,30475,5.591883e+06
2,30476,5.591883e+06
3,30477,5.591883e+06
4,30478,5.591883e+06


In [ ]:
# save result
if not os.path.exists('predict'):
    os.makedirs('predict')
df.to_csv('predict/mlp_predict.csv', index=False)
df_selected.to_csv('predict/mlp_selected_predict.csv', index=False)